In [1]:
import pandas as pd

In [2]:
# import Bexar cases dataframe
bexar_cases = pd.read_csv('Bexar_County_COVID-19_Data_by_Zip_Code.csv', index_col=0)
bexar_cases.columns = bexar_cases.columns.str.lower()
bexar_cases.head()

,zip_code,populationtotals_totpop_cy,positive,casesp100000,zipint,activecases,activecaserate,shape_length,shape_area
OBJECTID,,,,,,,,,
1,78002,9061,378,4171.724975,78002,24,264.871427,0.427542,0.009546
2,78006,5243,96,1831.012779,78006,7,133.511348,0.552725,0.005416
4,78015,12254,192,1566.835319,78015,16,130.569610,0.278955,0.002312
5,78023,29569,642,2171.192803,78023,49,165.714092,0.886455,0.017922
7,78052,699,28,4005.722461,78052,1,143.061516,0.260085,0.001147


In [3]:
bexar_cases.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74 entries, 1 to 86
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   zip_code                    74 non-null     int64  
 1   populationtotals_totpop_cy  74 non-null     int64  
 2   positive                    74 non-null     int64  
 3   casesp100000                74 non-null     float64
 4   zipint                      74 non-null     int64  
 5   activecases                 74 non-null     int64  
 6   activecaserate              74 non-null     float64
 7   shape_length                74 non-null     float64
 8   shape_area                  74 non-null     float64
dtypes: float64(4), int64(5)
memory usage: 5.8 KB


In [4]:
# how many unique zip codes are in the cases df?
bexar_cases.nunique()

zip_code                      74
populationtotals_totpop_cy    74
positive                      73
casesp100000                  74
zipint                        74
activecases                   53
activecaserate                74
shape_length                  74
shape_area                    74
dtype: int64

In [5]:
# import san antonio svi index dataframe
svidf = pd.read_csv('san_antonio_2018_tract.csv')
svidf.columns = svidf.columns.str.lower()
svidf = svidf.rename(columns={'fips': 'tract'})
svidf.head()

,st,state,st_abbr,stcnty,county,tract,location,area_sqmi,e_totpop,m_totpop,...,f_crowd,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,0,1,1,3,3,272,130,11.3,5.0,38328
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,0,1,0,2,4,606,206,20.0,5.7,9202
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,0,1,0,1,9,465,125,19.5,4.4,2044
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,0,1,1,3,8,690,265,24.5,7.4,6776
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,0,1,0,1,3,251,128,22.5,8.8,3716


In [6]:
# how many unique FIPS ids are in the svi df?
svidf.tract.value_counts()

48029181505    1
48029181602    1
48029181818    1
48029151900    1
48029182105    1
              ..
48029131601    1
48029191504    1
48029181726    1
48029191506    1
48029121112    1
Name: tract, Length: 366, dtype: int64

In [10]:
# get list of unique zip codes in the bexar cases df
bexar_zip = bexar_cases.zip_code.tolist()

In [11]:
# import track to zip dataframe
zips = pd.read_csv('TRACT_ZIP_122018_78s_only.csv')
# filter the zips df to only those in the bexar zip list
zips = zips[zips.zip.isin(bexar_zip)]
zips

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
14,48013960100,78264,0.00,0.00,0.00,0.00
16,48013960201,78052,0.63,0.88,0.67,0.64
17,48013960201,78069,0.19,0.07,0.13,0.18
20,48013960201,78002,0.00,0.00,0.00,0.00
22,48013960202,78069,0.19,0.11,0.48,0.20
...,...,...,...,...,...,...
2815,48493000103,78152,0.01,0.01,0.04,0.01
2818,48493000201,78112,0.07,0.00,0.19,0.08
2820,48493000201,78223,0.01,0.00,0.00,0.01
2827,48493000403,78101,0.48,0.14,0.33,0.47


In [12]:
zips[zips.zip == 78219]

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
164,48029120100,78219,0.00,0.00,0.00,0.00
169,48029120501,78219,0.00,0.03,0.01,0.00
171,48029120502,78219,0.00,0.01,0.00,0.00
222,48029121300,78219,0.00,0.01,0.04,0.00
298,48029130800,78219,0.48,0.76,0.27,0.51
300,48029130900,78219,0.84,0.95,0.91,0.86
303,48029131000,78219,0.00,0.01,0.05,0.00
313,48029131503,78219,1.00,1.00,1.00,1.00
314,48029131504,78219,1.00,1.00,1.00,1.00
317,48029131506,78219,0.14,0.77,0.06,0.17


In [13]:
# requiring the total ratio to be 54% or higher per track gives the closest to 366 without going over
# there will be 4 zip codes not represented and 2 census tracks not represented
# when merged we should get null values for these
zip_54 = zips[zips.tot_ratio >= .54]
zip_54.nunique()

tract        364
zip           70
res_ratio     47
bus_ratio     78
oth_ratio     68
tot_ratio     46
dtype: int64

In [14]:
# create df with only values to merge
merge_zip = zip_54[['tract', 'zip']]
merge_zip

,tract,zip
16,48013960201,78052
30,48013960401,78264
143,48029110300,78210
147,48029110500,78207
148,48029110600,78207
...,...,...
1676,48259970301,78006
1678,48259970302,78006
1680,48259970401,78006
1683,48259970402,78006


In [15]:
# create new df merging svi and merge zip on tract
fulldf = pd.merge(svidf, merge_zip, on='tract', how='left')
fulldf.shape

(366, 125)

In [16]:
# will need to convert zip to int
type(fulldf.zip[0])

numpy.float64

In [17]:
# there are 21 tract ids that do not have a zip code
fulldf.isna().sum()

st             0
state          0
st_abbr        0
stcnty         0
county         0
              ..
m_uninsur      0
ep_uninsur     0
mp_uninsur     0
e_daypop       0
zip           21
Length: 125, dtype: int64

In [18]:
# create a df of just the row with a null zip code
missing_zip = fulldf[fulldf.zip.isna()]

In [19]:
missing_zip

,st,state,st_abbr,stcnty,county,tract,location,area_sqmi,e_totpop,m_totpop,...,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,NaN
32,48,TEXAS,TX,48029,Bexar,48029121203,"Census Tract 1212.03, Bexar County, Texas",2.886395,4390,532,...,0,0,0,0,1207,457,27.5,8.8,10084,NaN
34,48,TEXAS,TX,48029,Bexar,48029121205,"Census Tract 1212.05, Bexar County, Texas",1.134232,4467,570,...,1,0,1,3,736,200,16.5,4.7,6334,NaN
63,48,TEXAS,TX,48029,Bexar,48029121905,"Census Tract 1219.05, Bexar County, Texas",3.045696,6039,637,...,0,0,0,0,433,251,7.2,4.1,5913,NaN
72,48,TEXAS,TX,48029,Bexar,48029130402,"Census Tract 1304.02, Bexar County, Texas",0.999486,4259,515,...,1,0,2,9,756,353,17.8,7.8,1678,NaN
73,48,TEXAS,TX,48029,Bexar,48029130500,"Census Tract 1305, Bexar County, Texas",0.882004,4812,559,...,1,0,1,6,1197,336,24.9,6.3,5110,NaN
76,48,TEXAS,TX,48029,Bexar,48029130800,"Census Tract 1308, Bexar County, Texas",3.278446,5911,717,...,1,0,1,5,1279,336,21.7,4.9,12144,NaN
99,48,TEXAS,TX,48029,Bexar,48029131700,"Census Tract 1317, Bexar County, Texas",4.532784,1182,174,...,0,0,0,2,44,46,4.7,4.6,3793,NaN
101,48,TEXAS,TX,48029,Bexar,48029131802,"Census Tract 1318.02, Bexar County, Texas",48.068125,4556,472,...,0,0,1,1,707,355,15.5,6.7,5014,NaN
122,48,TEXAS,TX,48029,Bexar,48029141900,"Census Tract 1419, Bexar County, Texas",46.608137,4769,651,...,0,0,1,1,729,423,15.3,8.0,7297,NaN


In [20]:
# here is a list of the tracts missing zip codes
missing_zip.tract.value_counts()

48029131802    1
48029182103    1
48029980004    1
48029130402    1
48029141900    1
48029191503    1
48029181704    1
48029131700    1
48029121205    1
48029110100    1
48029182102    1
48029192100    1
48029121905    1
48029130800    1
48029121203    1
48029171602    1
48029181725    1
48029150300    1
48029191504    1
48029191004    1
48029130500    1
Name: tract, dtype: int64

In [21]:
# find value for 48029131802
zips[zips.tract == 48029131802]
# assign 78101 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
345,48029131802,78101,0.34,0.23,0.47,0.34
346,48029131802,78263,0.24,0.28,0.32,0.25
347,48029131802,78109,0.26,0.24,0.12,0.25
348,48029131802,78152,0.10,0.02,0.04,0.09
349,48029131802,78220,0.05,0.18,0.04,0.06
350,48029131802,78219,0.01,0.05,0.02,0.01


In [22]:
# find value for 48029182103
zips[zips.tract == 48029182103]
# assign 78023 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
732,48029182103,78023,0.41,0.89,0.71,0.42
733,48029182103,78255,0.39,0.04,0.22,0.38
734,48029182103,78006,0.20,0.07,0.06,0.20


In [23]:
# find value for 48029980004
zips[zips.tract == 48029980004]
# assign 78216 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
858,48029980004,78216,0.0,0.20,0.9,0.12
859,48029980004,78247,0.0,0.09,0.0,0.05


In [24]:
# find value for 48029130402
zips[zips.tract == 48029130402]
# assign 78210 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
291,48029130402,78210,0.53,0.57,0.19,0.51
292,48029130402,78220,0.44,0.40,0.81,0.46
293,48029130402,78203,0.03,0.03,0.00,0.03


In [25]:
# find value for 48029141900
zips[zips.tract == 48029141900]
# assign 78263 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
381,48029141900,78263,0.49,0.55,0.45,0.49
382,48029141900,78101,0.25,0.20,0.28,0.25
383,48029141900,78223,0.23,0.24,0.23,0.23
384,48029141900,78222,0.03,0.01,0.04,0.03


In [26]:
# find value for 48029191503
zips[zips.tract == 48029191503]
# assign 78249 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
794,48029191503,78249,0.41,0.36,0.92,0.41
796,48029191503,78230,0.27,0.00,0.05,0.21


In [27]:
# find value for 48029181704
zips[zips.tract == 48029181704]
# assign 78238 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
646,48029181704,78238,0.47,0.51,0.60,0.49
648,48029181704,78250,0.14,0.18,0.39,0.15


In [28]:
# find value for 48029131700
zips[zips.tract == 48029131700]
# assign 78148 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
340,48029131700,78148,0.75,0.0,0.14,0.29


In [29]:
# find value for 48029121205
zips[zips.tract == 48029121205]
# assign 78233 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
215,48029121205,78233,0.67,0.03,0.26,0.52
216,48029121205,78217,0.33,0.97,0.74,0.48


In [30]:
# find value for 48029110100
zips[zips.tract == 48029110100]
# assign 78205 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
126,48029110100,78205,0.39,0.66,0.64,0.52
127,48029110100,78207,0.16,0.07,0.09,0.12
128,48029110100,78215,0.07,0.06,0.11,0.06
129,48029110100,78204,0.05,0.03,0.06,0.04
139,48029110100,78212,0.01,0.01,0.04,0.01
141,48029110100,78203,0.00,0.00,0.00,0.00
142,48029110100,78202,0.00,0.00,0.00,0.00


In [31]:
# find value for 48029182102
zips[zips.tract == 48029182102]
# assign 78255 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
728,48029182102,78255,0.51,0.08,0.03,0.50
729,48029182102,78006,0.42,0.63,0.58,0.43
730,48029182102,78257,0.06,0.27,0.35,0.07
731,48029182102,78015,0.00,0.01,0.03,0.00


In [32]:
# find value for 48029192100
zips[zips.tract == 48029192100]
# assign 78204 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
840,48029192100,78204,0.32,0.27,0.40,0.31
842,48029192100,78210,0.21,0.11,0.52,0.21
843,48029192100,78207,0.15,0.14,0.00,0.14
844,48029192100,78205,0.01,0.11,0.07,0.03


In [33]:
# find value for 48029121905
zips[zips.tract == 48029121905]
# assign 78259 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
264,48029121905,78259,0.54,0.49,0.52,0.53
265,48029121905,78261,0.46,0.20,0.21,0.45
266,48029121905,78258,0.00,0.31,0.27,0.02


In [34]:
# find value for 48029130800
zips[zips.tract == 48029130800]
# assign 78219 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
298,48029130800,78219,0.48,0.76,0.27,0.51
299,48029130800,78220,0.52,0.24,0.73,0.49


In [35]:
# find value for 48029121203
zips[zips.tract == 48029121203]
# assign 78233 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
207,48029121203,78233,0.30,0.25,0.51,0.29
208,48029121203,78217,0.26,0.19,0.40,0.24
210,48029121203,78218,0.00,0.01,0.07,0.01


In [36]:
# find value for 48029171602
zips[zips.tract == 48029171602]
# assign 78227 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
534,48029171602,78227,0.51,0.83,0.33,0.53
535,48029171602,78237,0.49,0.17,0.67,0.47


In [37]:
# find value for 48029181725
zips[zips.tract == 48029181725]
# assign 78251 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
671,48029181725,78251,0.56,0.17,0.33,0.53
672,48029181725,78238,0.44,0.83,0.67,0.47


In [38]:
# find value for 48029150300
zips[zips.tract == 48029150300]
# assign 78214 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
386,48029150300,78214,0.46,0.46,0.51,0.47
387,48029150300,78204,0.38,0.49,0.47,0.40
388,48029150300,78210,0.15,0.05,0.02,0.14


In [39]:
# find value for 48029191504
zips[zips.tract == 48029191504]
# assign 78231 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
798,48029191504,78231,0.52,0.16,0.11,0.44
799,48029191504,78249,0.25,0.52,0.79,0.32
800,48029191504,78230,0.23,0.32,0.11,0.24


In [40]:
# find value for 48029191004
zips[zips.tract == 48029191004]
# assign 78216 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
759,48029191004,78216,0.50,0.60,0.60,0.52
760,48029191004,78213,0.47,0.02,0.23,0.41
761,48029191004,78212,0.03,0.37,0.17,0.07


In [41]:
# find value for 48029130500
zips[zips.tract == 48029130500]
# assign 78202 to this track = highest % addresses

,tract,zip,res_ratio,bus_ratio,oth_ratio,tot_ratio
294,48029130500,78202,0.55,0.23,0.4,0.52
295,48029130500,78203,0.45,0.77,0.6,0.48


In [42]:
tracts_list = missing_zip.tract.tolist()

In [43]:
assign_zip_list = [78101, 78023, 78216, 78210, 78263, 78249, 78238, 78148, 78233, 78205, 78255, 78204, 78259,
                   78219, 78233, 78227, 78251, 78214, 78231, 78216, 78202]

In [44]:
len(tracts_list)

21

In [45]:
len(assign_zip_list)

21

In [46]:
# verified lists are same length, now zip lists together into df to merge back to fulldf
# https://cmdlinetips.com/2018/01/how-to-create-pandas-dataframe-from-multiple-lists/#:~:text=Create%20pandas%20dataframe%20from%20lists%20using%20zip%20Second,it%20to%20produce%20one%20item%20at%20a%20time.
data_to_add = pd.DataFrame()
data_to_add['tract'] = tracts_list
data_to_add['zip'] = assign_zip_list
data_to_add

,tract,zip
0,48029110100,78101
1,48029121203,78023
2,48029121205,78216
3,48029121905,78210
4,48029130402,78263
5,48029130500,78249
6,48029130800,78238
7,48029131700,78148
8,48029131802,78233
9,48029141900,78205


In [47]:
# add these values to the merge zip df
merge_zip2 = merge_zip.append(data_to_add)

In [48]:
len(merge_zip2)

385

In [49]:
# create new df merging svi and 2nd merge zip file on tract
svi_zip = pd.merge(svidf, merge_zip2, on='tract', how='left')
svi_zip.shape

(366, 125)

In [50]:
# check for nulls
svi_zip.isna().sum()

st            0
state         0
st_abbr       0
stcnty        0
county        0
             ..
m_uninsur     0
ep_uninsur    0
mp_uninsur    0
e_daypop      0
zip           0
Length: 125, dtype: int64

In [51]:
svi_zip.head()

,st,state,st_abbr,stcnty,county,tract,location,area_sqmi,e_totpop,m_totpop,...,f_noveh,f_groupq,f_theme4,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,1,1,3,3,272,130,11.3,5.0,38328,78101
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,1,0,2,4,606,206,20.0,5.7,9202,78210
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,1,0,1,9,465,125,19.5,4.4,2044,78207
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,1,1,3,8,690,265,24.5,7.4,6776,78207
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,1,0,1,3,251,128,22.5,8.8,3716,78212


In [52]:
bexar_cases.head()

,zip_code,populationtotals_totpop_cy,positive,casesp100000,zipint,activecases,activecaserate,shape_length,shape_area
OBJECTID,,,,,,,,,
1,78002,9061,378,4171.724975,78002,24,264.871427,0.427542,0.009546
2,78006,5243,96,1831.012779,78006,7,133.511348,0.552725,0.005416
4,78015,12254,192,1566.835319,78015,16,130.569610,0.278955,0.002312
5,78023,29569,642,2171.192803,78023,49,165.714092,0.886455,0.017922
7,78052,699,28,4005.722461,78052,1,143.061516,0.260085,0.001147


In [57]:
# rename the zip code column to merge on
bexar_cases = bexar_cases.rename(columns={'zip_code': 'zip', 'populationtotals_totpop_cy': 'population'})

In [58]:
bexar_cases

,zip,population,positive,casesp100000,zipint,activecases,activecaserate,shape_length,shape_area
OBJECTID,,,,,,,,,
1,78002,9061,378,4171.724975,78002,24,264.871427,0.427542,0.009546
2,78006,5243,96,1831.012779,78006,7,133.511348,0.552725,0.005416
4,78015,12254,192,1566.835319,78015,16,130.569610,0.278955,0.002312
5,78023,29569,642,2171.192803,78023,49,165.714092,0.886455,0.017922
7,78052,699,28,4005.722461,78052,1,143.061516,0.260085,0.001147
...,...,...,...,...,...,...,...,...,...
82,78260,36255,786,2167.976831,78260,64,176.527376,0.407156,0.005818
83,78261,23828,507,2127.748867,78261,50,209.837166,0.474302,0.006743
84,78263,5228,138,2639.632747,78263,8,153.022188,0.680812,0.010405


In [59]:
# create dataframe to merge with svi
merge_bexar = bexar_cases[['zip', 'population', 'positive', 'casesp100000']]
merge_bexar.head()

,zip,population,positive,casesp100000
OBJECTID,,,,
1,78002,9061,378,4171.724975
2,78006,5243,96,1831.012779
4,78015,12254,192,1566.835319
5,78023,29569,642,2171.192803
7,78052,699,28,4005.722461


In [60]:
# merge svi_zip with merge_bexar
svi_zip_cases = pd.merge(svi_zip, merge_bexar, on='zip', how='left')
svi_zip_cases.head()

,st,state,st_abbr,stcnty,county,tract,location,area_sqmi,e_totpop,m_totpop,...,f_total,e_uninsur,m_uninsur,ep_uninsur,mp_uninsur,e_daypop,zip,population,positive,casesp100000
0,48,TEXAS,TX,48029,Bexar,48029110100,"Census Tract 1101, Bexar County, Texas",1.364296,3102,365,...,3,272,130,11.3,5.0,38328,78101,4485,104,2318.840580
1,48,TEXAS,TX,48029,Bexar,48029110300,"Census Tract 1103, Bexar County, Texas",0.598444,3023,309,...,4,606,206,20.0,5.7,9202,78210,39242,1583,4033.943224
2,48,TEXAS,TX,48029,Bexar,48029110500,"Census Tract 1105, Bexar County, Texas",0.448810,2388,243,...,9,465,125,19.5,4.4,2044,78207,58019,3205,5524.052466
3,48,TEXAS,TX,48029,Bexar,48029110600,"Census Tract 1106, Bexar County, Texas",0.758136,5301,486,...,8,690,265,24.5,7.4,6776,78207,58019,3205,5524.052466
4,48,TEXAS,TX,48029,Bexar,48029110700,"Census Tract 1107, Bexar County, Texas",0.379005,1114,208,...,3,251,128,22.5,8.8,3716,78212,29916,966,3229.041316


In [61]:
# check for nulls
svi_zip_cases.isna().sum()

st              0
state           0
st_abbr         0
stcnty          0
county          0
               ..
e_daypop        0
zip             0
population      0
positive        0
casesp100000    0
Length: 128, dtype: int64

In [62]:
# write final compliation to csv
#svi_zip_cases.to_csv('full_san_antonio.csv')